<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=933c1b502db2dbc0f6f8d30151ee8053542663ae107c6f63e1a2a2053ff81592
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-11 15:19:50
-------------------
qualified stocks: 86
with latest results: 24
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  99.50 K
Current:  1.43 C
-------------------
Today PnL: 29.85 K (0.21%)
Current PnL: -21.91 L (-14.35%)
CY Booked + Current PnL: -8.72 L (-5.71%)
-------------------
Total profit:  1.68 L
Total loss:  -23.59 L
-------------------
Total Booked + Current PnL: 18.26 L (14.54%)
Total Booked PnL: 40.17 L (31.98%)
Curr Year Booked PnL: 13.19 L (9.21%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 89.37 L (62.4%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.63%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,100.85,50.0,M-SC,2.34,86499.0,-14278.0,14367.0,-0.05,-14.17,16.61,0.09,245.0,-0.99,0.61,13.64,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.05,48.0,X-MC,3.14,88905.0,-1773.0,16679.0,0.44,-1.96,18.76,16.44,101.0,-0.11,0.63,17.83,X40,ATH,PHARMA
42,ITC,452.00,-38.34,46.0,X-LC,0.83,198876.0,-1262.0,22155.0,0.28,-0.63,11.14,10.44,4.0,-0.06,1.40,4.90,X40,NTT,FMCG
34,ICICIGI,2252.93,-14.07,54.0,X-MC,6.59,185842.0,11522.0,23676.0,-1.34,6.61,12.74,20.19,91.0,0.49,1.31,22.72,X40,ATH,INSURANCE
51,NESTLEIND,1377.00,-3.51,60.0,X-LC,8.23,296735.0,31309.0,25490.0,0.41,11.80,8.59,21.40,11.0,1.23,2.09,19.22,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3586.49,52.0,M-SC,3.81,165512.0,-31620.0,121453.0,9.21,-16.04,73.38,45.57,236.0,-0.26,1.16,17.34,XY24,NTT,AUTO
39,INDUSINDBK,1800.00,283.05,71.0,L-MC,7.38,47051.0,-30156.0,55548.0,3.27,-39.06,118.06,32.89,258.0,-0.54,0.33,29.57,XR,NTT,BANKS
38,INDIGOPNTS,1408.00,125.45,77.0,M-SC,5.28,147870.0,-26689.0,26720.0,3.22,-15.29,18.07,0.02,221.0,-1.00,1.04,28.62,OX40N,NTT,PAINTS
14,BSOFT,836.99,-13.25,61.0,H-SC,9.73,97538.0,-43115.0,115895.0,2.15,-30.65,118.82,51.74,131.0,-0.37,0.69,12.85,XR,ATH,IT
13,BERGEPAINT,680.00,-17.69,56.0,X-MC,1.09,221414.0,-5925.0,53981.0,2.07,-2.61,24.38,21.14,106.0,-0.11,1.56,24.80,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-45.08,37.0,M-SC,8.35,113786.0,-1012.0,95057.0,-3.77,-0.88,83.54,81.92,223.0,-0.01,0.80,34.70,XR,NTT,DURABLES
83,VOLTAS,1530.00,-6.67,36.0,X-MC,4.02,195705.0,3963.0,33798.0,-2.69,2.07,17.27,19.69,99.0,0.12,1.38,9.00,XY25,NTT,AC
54,QUESS,986.00,-52.07,31.0,X-SC,37.28,47781.0,-17225.0,168151.0,-2.56,-26.50,351.92,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
26,GREENPANEL,537.00,197.80,37.0,M-SC,5.24,132907.0,-50171.0,129146.0,-2.42,-27.40,97.17,43.14,230.0,-0.39,0.93,25.02,XY24,NTT,MISC
25,GILLETTE,11206.78,-19.32,16.0,X-MC,5.55,248160.0,-5736.0,88047.0,-2.41,-2.26,35.48,32.42,105.0,-0.07,1.74,12.50,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-45.08,37.0,M-SC,8.35,113786.0,-1012.0,95057.0,-3.77,-0.88,83.54,81.92,223.0,-0.01,0.8,34.7,XR,NTT,DURABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,100.85,50.0,M-SC,2.34,86499.0,-14278.0,14367.0,-0.05,-14.17,16.61,0.09,245.0,-0.99,0.61,13.64,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.0,125.45,77.0,M-SC,5.28,147870.0,-26689.0,26720.0,3.22,-15.29,18.07,0.02,221.0,-1.00,1.04,28.62,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-68.16,40.0,H-SC,2.26,219078.0,-50589.0,86930.0,-0.98,-18.76,39.68,13.47,138.0,-0.58,1.54,11.39,XY24,NTT,PAINTS
67,SIS,528.0,2012.95,45.0,H-SC,2.23,85272.0,-25760.0,49364.0,-0.10,-23.20,57.89,21.26,156.0,-0.52,0.60,15.21,OX40N,NTT,MISC
43,JCHAC,2282.0,19943.48,63.0,M-SC,1.49,97944.0,-29861.0,29844.0,0.04,-23.36,30.47,-0.01,232.0,-1.00,0.69,10.91,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,57.18,70.0,M-MC,10.49,248295.0,23333.0,144309.0,-0.10,10.37,58.12,74.53,192.0,0.16,1.75,45.97,XY24,BTT,STEEL
32,HINDZINC,730.22,29.80,55.0,M-LC,9.20,211954.0,6878.0,105701.0,0.55,3.35,49.87,54.89,52.0,0.07,1.49,28.26,X5K,ATH,METALS
37,INDIAMART,4810.62,-53.52,52.0,H-SC,5.08,129850.0,6514.0,125110.0,-0.07,5.28,96.35,106.72,119.0,0.05,0.91,29.89,AR,ATH,MISC
62,SAMMAANCAP,326.00,-192.05,53.0,M-SC,35.14,158823.0,9603.0,134571.0,-1.55,6.44,84.73,96.62,208.0,0.07,1.12,71.88,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,29.80,55.0,M-LC,9.20,211954.0,6878.0,105701.0,0.55,3.35,49.87,54.89,52.0,0.07,1.49,28.26,X5K,ATH,METALS
37,INDIAMART,4810.62,-53.52,52.0,H-SC,5.08,129850.0,6514.0,125110.0,-0.07,5.28,96.35,106.72,119.0,0.05,0.91,29.89,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-45.08,37.0,M-SC,8.35,113786.0,-1012.0,95057.0,-3.77,-0.88,83.54,81.92,223.0,-0.01,0.80,34.70,XR,NTT,DURABLES
85,WIPRO,420.00,-15.15,45.0,M-LC,5.60,148701.0,-2244.0,111704.0,1.42,-1.49,75.12,72.51,53.0,-0.02,1.05,4.45,XR,NTT,IT
47,KPIGREEN,731.05,2.88,44.0,H-SC,9.79,122699.0,-2598.0,61521.0,0.54,-2.07,50.14,47.03,141.0,-0.04,0.86,53.63,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GILLETTE,11206.78,-19.32,16.0,X-MC,5.55,248160.0,-5736.0,88047.0,-2.41,-2.26,35.48,32.42,105.0,-0.07,1.74,12.50,X40,ATH,FMCG
74,TATAMOTORS,1065.00,-53.17,24.0,X-LC,36.18,148993.0,-127530.0,237599.0,1.17,-46.12,159.47,39.81,54.0,-0.54,1.05,5.01,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.49,86951.0,-41719.0,87021.0,-0.18,-32.42,100.08,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-32.66,31.0,X-MC,0.69,195000.0,-5820.0,73612.0,-2.14,-2.90,37.75,33.76,80.0,-0.08,1.37,2.15,X40,ATH,FMCG
54,QUESS,986.00,-52.07,31.0,X-SC,37.28,47781.0,-17225.0,168151.0,-2.56,-26.50,351.92,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PGHH,17907.65,-32.66,31.0,X-MC,0.69,195000.0,-5820.0,73612.0,-2.14,-2.90,37.75,33.76,80.0,-0.08,1.37,2.15,X40,ATH,FMCG
42,ITC,452.00,-38.34,46.0,X-LC,0.83,198876.0,-1262.0,22155.0,0.28,-0.63,11.14,10.44,4.0,-0.06,1.40,4.90,X40,NTT,FMCG
66,SIEMENS,4671.50,-6.04,37.0,H-LC,1.62,151165.0,-34930.0,82415.0,-1.02,-18.77,54.52,25.51,15.0,-0.42,1.06,11.40,AR,ATH,ELECTRICAL
29,HAVELLS,2069.16,-9.16,36.0,X-MC,3.31,303324.0,-24667.0,131188.0,-0.91,-7.52,43.25,32.48,92.0,-0.19,2.13,3.05,X40,ATH,ELECTRICAL
79,UNITDSPR,1644.00,-7.24,59.0,X-LC,3.62,243328.0,9184.0,39443.0,0.37,3.92,16.21,20.77,86.0,0.23,1.71,11.11,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.49,86951.0,-41719.0,87021.0,-0.18,-32.42,100.08,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
54,QUESS,986.00,-52.07,31.0,X-SC,37.28,47781.0,-17225.0,168151.0,-2.56,-26.50,351.92,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
1,ABB,7934.00,-42.78,32.0,H-LC,4.98,239064.0,-22555.0,141765.0,-0.23,-8.62,59.30,45.57,7.0,-0.16,1.68,1.72,AR,NTT,ELECTRICAL
10,BAJAJHFL,181.50,-16.17,36.0,X-MC,6.36,176628.0,-23878.0,121026.0,-0.52,-11.91,68.52,48.45,90.0,-0.20,1.24,1.85,X40N,ATH,FINANCE
52,PAGEIND,51769.93,-28.87,39.0,X-MC,7.63,160800.0,-2940.0,46278.0,0.00,-1.80,28.78,26.47,82.0,-0.06,1.13,1.88,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-52.07,31.0,X-SC,37.28,47781.0,-17225.0,168151.0,-2.56,-26.50,351.92,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-40.65,42.0,X-SC,4.03,80943.0,-64217.0,140145.0,0.61,-44.24,173.14,52.31,136.0,-0.46,0.57,8.35,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.49,86951.0,-41719.0,87021.0,-0.18,-32.42,100.08,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.05,48.0,X-MC,3.14,88905.0,-1773.0,16679.0,0.44,-1.96,18.76,16.44,101.0,-0.11,0.63,17.83,X40,ATH,PHARMA
33,HONAUT,58357.33,-24.48,35.0,X-SC,0.93,106485.0,-21453.0,68587.0,-1.44,-16.77,64.41,36.84,143.0,-0.31,0.75,9.01,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-28.04,54.0,X-LC,9.34,286587.0,-59369.0,126070.0,0.78,-17.16,43.99,19.28,1.0,-0.47,2.02,5.95,X40,ATH,IT
40,INFY,2275.00,-16.37,63.0,X-LC,4.70,325656.0,12768.0,158920.0,1.02,4.08,48.80,54.87,3.0,0.08,2.29,12.70,X40,BTT,IT
42,ITC,452.00,-38.34,46.0,X-LC,0.83,198876.0,-1262.0,22155.0,0.28,-0.63,11.14,10.44,4.0,-0.06,1.40,4.90,X40,NTT,FMCG
82,VBL,671.64,-16.73,53.0,X-LC,5.74,299860.0,-15982.0,128640.0,1.46,-5.06,42.90,35.67,5.0,-0.12,2.11,8.09,X40N,ATH,FMCG
1,ABB,7934.00,-42.78,32.0,H-LC,4.98,239064.0,-22555.0,141765.0,-0.23,-8.62,59.30,45.57,7.0,-0.16,1.68,1.72,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,41.0,L-SC,27.23,76512.0,-37037.0,77086.0,-1.94,-32.62,100.75,35.27,268.0,-0.48,0.54,87.27,XR,NTT,HOTELS
7,ASIANTILES,137.00,7572.22,68.0,L-SC,14.07,84169.0,-9641.0,86256.0,0.10,-10.28,102.48,81.67,269.0,-0.11,0.59,63.00,XR,NTT,CERAMICS
50,MASFIN,398.61,-20.05,46.0,H-SC,7.91,90915.0,-7065.0,28665.0,-0.64,-7.21,31.53,22.05,152.0,-0.25,0.64,32.15,XR,ATH,FINANCE
47,KPIGREEN,731.05,2.88,44.0,H-SC,9.79,122699.0,-2598.0,61521.0,0.54,-2.07,50.14,47.03,141.0,-0.04,0.86,53.63,MH,ATH,POWER
80,VAIBHAVGBL,521.00,68.04,57.0,H-SC,5.63,145484.0,-37291.0,148365.0,-1.62,-20.40,101.98,60.77,125.0,-0.25,1.02,31.02,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7572.22,68.0,L-SC,14.07,84169.0,-9641.0,86256.0,0.10,-10.28,102.48,81.67,269.0,-0.11,0.59,63.00,XR,NTT,CERAMICS
80,VAIBHAVGBL,521.00,68.04,57.0,H-SC,5.63,145484.0,-37291.0,148365.0,-1.62,-20.40,101.98,60.77,125.0,-0.25,1.02,31.02,XR,NTT,JEWELLERY
38,INDIGOPNTS,1408.00,125.45,77.0,M-SC,5.28,147870.0,-26689.0,26720.0,3.22,-15.29,18.07,0.02,221.0,-1.00,1.04,28.62,OX40N,NTT,PAINTS
6,ASIANPAINT,3465.66,-7.31,75.0,X-LC,6.19,234010.0,-17758.0,70975.0,0.33,-7.05,30.33,21.13,27.0,-0.25,1.65,25.87,X40,ATH,PAINTS
5,ANGELONE,3033.00,22.45,68.0,X-SC,6.67,216390.0,25384.0,32307.0,1.25,13.29,14.93,30.21,157.0,0.79,1.52,36.95,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.89
1,25,44.83
2,50,76.26


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.52
LC    30.52
MC    25.99
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.87
X40      19.30
X40N     12.45
XY25     10.23
XR        9.70
AR        7.71
OX40N     6.60
X200      1.81
X5K       1.49
SR        1.01
MH        0.86
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.91
X-MC    21.57
X-LC    21.03
M-SC    12.58
X-SC     5.57
H-LC     4.55
M-LC     3.94
H-MC     2.34
M-MC     1.75
L-SC     1.46
L-LC     1.00
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.00,-4.10,37.76
IT,12.67,-16.87,79.42
FINANCE,12.12,-9.61,59.30
MISC,7.69,-18.80,78.72
PAINTS,5.79,-12.28,29.01
ELECTRICAL,5.62,-12.95,52.99
INSURANCE,4.48,0.54,34.51
AUTO,2.81,-43.60,103.53
BANKS,2.71,-38.69,129.08


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3284304.0,21
XR,1279078.0,13
AR,1189792.0,9
X40,948387.0,13
X40N,696526.0,9
OX40N,546529.0,9
XY25,481546.0,7
SR,276744.0,2
X5K,105701.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3430780.0,24
X-MC,1342062.0,15
M-SC,1255872.0,15
X-LC,1050819.0,12
X-SC,556983.0,6
H-LC,290952.0,3
M-LC,263388.0,3
H-MC,255485.0,2
L-SC,252965.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1187317.0      6
           AR         841503.0      5
           XR         804224.0      7
M-SC       XY24       729141.0      6
X-MC       XY24       565250.0      4
X-LC       X40        433425.0      5
X-MC       X40        427941.0      7
X-LC       XY24       308295.0      2
X-SC       X40N       301811.0      4
H-SC       SR         276744.0      2
           OX40N      259471.0      3
X-LC       X40N       234098.0      3
H-LC       AR         224180.0      2
X-MC       XY25       188254.0      2
H-MC       XY24       181841.0      1
X-SC       XY24       168151.0      1
L-SC       XR         163342.0      2
X-MC       X40N       160617.0      2
M-MC       XY24       144309.0      1
M-SC       XR         144260.0      2
           XY25       134571.0      1
           AR         124109.0      2
           OX40N      123791.0      4
M-LC       XR         111704.0      1
           X5K        105701.0      1
L-SC       OX40N       89623.0      1
X-SC       X40         87021.0      1
X-LC       XY25        75001.0      2
H-MC       OX40N       73644.0      1
H-LC       X200        66772.0      1
H-SC       MH          61521.0      1
L-MC       XR          55548.0      1
M-LC       XY25        45983.0      1
L-LC       XY25        37737.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 40.0 seconds
